In [5]:
import pandas as pd
import numpy as np

import time
from datetime import datetime

from random import randint

import re
import string

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer

from sklearn.utils import shuffle

from tqdm import tqdm

import json

import paralleldots
paralleldots.set_api_key("sHyni9kN5Elhtrmb3Z10f5USPyyn9o0snPCzIqagafM") # laualan@hku.hk
print('Current key:', 'sHyni9kN5Elhtrmb3Z10f5USPyyn9o0snPCzIqagafM', 1)


keys = [
        'DwGe3alGduDGJnE1FT3O112wMDNJhZDnGysR0KFfZgY', # rlalan@outlook.com
        'JIkATWugt8LIP3PbQrdm6cfm1hZ4DbouVoyw8oAQqhI', # 394414515@qq.com
        'Vf9mBAl7wp0s8Fl43E9aDtVPOZhftIAv0pYJvIlYGD4', # 806124854@qq.com
        '8gAwuDOEE92zakGfDXc6PoCtqhVop0htEnEd4IHFe0U', # yleun.lau@gmail.com
        'xTNXoCxtbg36jCgT6ArKgyOPJtnJdf5QQpEEKYoSwu8', # alanconstantinelau@gmail.com
        'INsEQXjCWuaZJQyzEbHg5Kw6C8liJiHf2DXOHqmHG70', # 122075300@qq.com
        'jpctHohVPtW9QlezbIw4hZ8Ftwh8kXD14U7JlR0WRQo', # 274038499@qq.com
        'qrJoLzvIQqU1ygqtzWDw2JC59zkBsPqSzLLIeRSxLHA', # 562040899@qq.com
        'UtEqIoCXWJN5XiyfgpXJ7gg23LgOu7jVQjl36Xg7UmQ', # wgj0905@hku.hk
        'sHyni9kN5Elhtrmb3Z10f5USPyyn9o0snPCzIqagafM' # liuhuan19951021@163.com
        ]

Current key: sHyni9kN5Elhtrmb3Z10f5USPyyn9o0snPCzIqagafM 1


In [18]:
df = pd.read_csv('./reindex_df.csv')
print(df.shape)

(237671, 7)


In [11]:
df.head()

,index,song,year,artist,genre,lyrics,word_count
0,0,louder-flux-pavilion-doctor-p-remix,2012,dj-fresh,Electronic,I can't control this feeling Something's happe...,341.0
1,1,that-s-my-name,2009,akcent,Pop,In my heart I will keep you In my heart Foreve...,318.0
2,2,lemonade,2007,apologetix,Rock,"Oh, Mama, I've been cleared of my crimes and I...",250.0
3,3,return-of-the-hustle,2007,fabolous,Hip-Hop,Record mode! (Record mode!) Get your money in ...,681.0
4,4,so-sad,1974,george-harrison,Rock,Now the winter has come To eclipse out the sun...,163.0


In [13]:
emo_path = r'./lyrics_emotion-fixed.json'

In [14]:
with open(emo_path, 'r') as fn:
    emo_dict = json.load(fn)
    
for k, v in tqdm(emo_dict.items()):
    if 'emotion' not in 'v':
        lyric = df.loc[df.index == int(k)]['lyrics']
        fix(lyric)
        break

In [19]:
def update_emo(index, emo):
    emo['time'] = str(datetime.now())
    try:
        with open(emo_path, 'r') as fn:
            emo_dict = json.load(fn)
        emo_dict[index] = emo
    except Exception as e:
        emo_dict = {}
        emo_dict[index] = emo
    with open(emo_path, 'w') as fp:
        json.dump(emo_dict, fp)
    return index

def update_error(index):
    with open(emo_path, 'a') as fn:
        fn.write(str(index) + '\n')

In [20]:
def fix(lyrics):
    try:
        time.sleep(randtime)
        response = paralleldots.emotion(lyrics)
        if 'code' in response and key_count < len(keys):
            key_count += 1
            print('Current key:', keys[key_count], key_count + 1)
            paralleldots.set_api_key(keys[key_count])
            response = paralleldots.emotion(lyrics)
        elif key_count == len(keys):
            print('Stop at', str(index))
            update_error(str(index) + '\tStop.')
            break
        elif 'message' in response:
            printt('Stop at', str(index))
            update_error(str(index) + '\t' + str(response['message']))
            break
        elif 'Error' in response:
            print('Stop at', str(index))
            update_error(str(index) + '\t' + str(response))
            break
        elif 'emotion' not in response:
            print('Stop at', str(index))
            update_error(str(index) + '\t' + str(response))
            break
        finished_index = update_emo(index, response)
#         print(finished_index, 'done: ', str(round(((index-1)/total)*100, 5)), str(datetime.now()))
    except Exception as e:
        update_error(str(index) + '\t' + str(e))
        error_count += 1
#         print(e)
        pass

SyntaxError: 'break' outside loop (<ipython-input-20-4acb00be7d1f>, line 16)

In [ ]:
with open(emo_path, 'w') as fp:
    json.dump(emo_dict, fp)